In [11]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [10]:
pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


# Load data and model

First load all the CSV files we will need

In [12]:
train = pd.read_csv("nlp-getting-started/train.csv")
test = pd.read_csv("nlp-getting-started/test.csv")
submission = pd.read_csv("nlp-getting-started/sample_submission.csv")

Create convenient names for the variables we will be using for training and inference.

In [22]:
train_data = train.text.values
train_labels = train.target.values
test_data = test.text.values

In [16]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
embed = hub.KerasLayer(module_url, trainable=True, name='USE_embedding')

# Train the model

In [17]:
def build_model(embed):
    
    model = Sequential([
        Input(shape=[], dtype=tf.string),
        embed,
        Dense(1, activation='sigmoid')
    ])
    model.compile(Adam(2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
model = build_model(embed)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_embedding (KerasLayer)   (None, 512)               147354880 
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 147,355,393
Trainable params: 147,355,393
Non-trainable params: 0
_________________________________________________________________


In [19]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

train_history = model.fit(
    train_data, train_labels,
    validation_split=0.25,
    epochs=4,
    callbacks=[checkpoint],
    batch_size=32
)

Epoch 1/4


179/179 [==============================] - 89s 297ms/step - loss: 0.5928 - accuracy: 0.7646 - val_loss: 0.4732 - val_accuracy: 0.8162
Epoch 2/4
179/179 [==============================] - 49s 276ms/step - loss: 0.4343 - accuracy: 0.8425 - val_loss: 0.4468 - val_accuracy: 0.8251
Epoch 3/4
179/179 [==============================] - 49s 272ms/step - loss: 0.3780 - accuracy: 0.8795 - val_loss: 0.4403 - val_accuracy: 0.8209
Epoch 4/4
179/179 [==============================] - 49s 272ms/step - loss: 0.3417 - accuracy: 0.9021 - val_loss: 0.4462 - val_accuracy: 0.8204


In [20]:
model.load_weights('model.h5')
test_pred = model.predict(test_data)

In [9]:
#write to submission file
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)